<a href="https://colab.research.google.com/github/Ken-huea/Language-processing/blob/main/%E2%91%A1%E8%87%AA%E7%84%B6%E8%A8%80%E8%AA%9E%E5%87%A6%E7%90%86%E3%81%A8%E3%83%99%E3%82%AF%E3%83%88%E3%83%AB%E8%A1%A8%E7%8F%BE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 文書のベクトル表現
文章の類似度を確認。複数の文章を数式化してどれとどれが近しいか。と。

単語の頻出や重みなどの手法で重要度は分かるだろうけど、それがどうベクトルに働くのか？

### イントロ

①形態素解析　⇒　②文章を数字に変換(ベクトル表現)　⇒　③numpyで比較

ベクトル表現に変換する方法

・**カウント表現**：文書中の<font color="red">**各単語の出現数**</font>に着目する方法

・**バイナリ表現**：出現頻度を気にせず、文章中に各単語が<font color="red">**出現したかどうか**</font>のみに着目する方法

・**tf-idf表現**：tf-idfという手法で計算された、文章中の<font color="red">**各単語の重み情報を扱う**</font>方法

<font color="red">**tf**</font>：重みを付ける事。

文章に｢犬｣が頻出したら、｢この犬という単語は重要なんだな｣と思うじゃん。

このように単語に重みを持たせる事。

だけど、助詞の「は」「が」「と」等はどこにでも頻出するので、文章の特徴とは言い難い。このような場合は重みを軽くする。みたいな。

## ベクトル表現①　Bag of Words(BOW)カウント表現
各単語の出現回数をカウントしベクトル化⇒<font color="red">不明質問</font>

例
- John likes to watch movies. Mary likes movies too.
- John also likes to watch football games.

`["John","likes","to","watch","movies","Mary","too","also","football", "games"]`  ⇒羅列して

1. [1, 2, 1, 1, 2, 1, 1, 0, 0, 0]　　⇒単語をカウントしてる likesとwatchが２回。⇒だから重要みたい。
2. [1, 1, 1, 1, 0, 0, 0, 1, 1, 1]


単語の出現回数は紛れもなく文章の特徴です。

欠点

・単語の語順情報がない
　・単語の意味の表現が苦手

この欠点を**Doc2Vec**は補っている。

In [ ]:
!pip install janome
# ライブラリ読み込み　今回はジャノメ
from janome.tokenizer import Tokenizer
from gensim import corpora

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.7 MB 6.5 MB/s 


###### 実装
[お手本コード](https://premium.aidemy.jp/courses/5050/exercises/rkUQc38sIxf)

例題：

・すもももももももものうち

・料理も景色もすばらしい

・私の趣味は写真撮影です

上記の3つの文を①janomeを使い形態素解析する。 　　②カウント表現 のBag of Wordsを作成し表示する。

In [ ]:
text1 = "すもももももももものうち"
text2 = "料理も景色もすばらしい"
text3 = "私の趣味は写真撮影です"

In [ ]:
t = Tokenizer()
# 分かち書き ⇒ wakati=True
tokens1 = t.tokenize(text1, wakati=True)
tokens2 = t.tokenize(text2, wakati=True)
tokens3 = t.tokenize(text3, wakati=True)

# tokens1 = t.tokenize(text1)
# tokens2 = t.tokenize(text2)
# tokens3 = t.tokenize(text3)

ジェネレーターなのでfor文で表示させる事。[詳細はこちら](https://qiita.com/keitakurita/items/5a31b902db6adfa45a70)

In [ ]:
print(type(tokens1))
for token in tokens1:
    print(token)

print()

print(type(tokens2))
for token in tokens2:
    print(token)

print()

print(type(tokens3))
for token in tokens3:
    print(token)

In [ ]:
# 上のコードを実装させるとリストが空になる…。
# 文章をまとめてリスト化し
documents = [tokens1, tokens2, tokens3]

# corporaを使い単語辞書を作成
dictionary = corpora.Dictionary(documents)

# 各単語のidを表示してください
print(dictionary.token2id)

{}


In [ ]:
# Bag of Wordsの作成
bow_corpus = [dictionary.doc2bow(d) for d in documents]

# (id, 出現回数)のリスト
print(bow_corpus)

print()
# bow_corpusの内容をわかりやすく出力する
texts = [text1, text2, text3]

for i in range(len(bow_corpus)):
    print(texts[i])
    for j in range(len(bow_corpus[i])):
        index = bow_corpus[i][j][0]
        num = bow_corpus[i][j][1]
        print("\"", dictionary[index], "\" が " ,num, "回", end=", ")
    print()

[[], [], []]

すもももももももものうち

料理も景色もすばらしい

私の趣味は写真撮影です



## ベクトル表現② tf-idf
特定の文書にのみ多く出現する単語⇒重要度が高くなる。

特定の文書にしか出現しない単語の重要度　　　⇒上げる。｢黄金｣ 「回転」 「レッスン」

多くの文書に出現する語（一般的な語）の重要度⇒下げる。「です」「ます」


ジョジョ７部は「黄金」「回転」という単語が特融で頻出する⇒<font color="red">重要度が高い</font>

### 実装
[こちらから](https://premium.aidemy.jp/courses/5050/exercises/HydX93IoUxz)

その①

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# 表示するときに有効数字２桁で表示する np.roundみたいに小数点を省略している。
np.set_printoptions(precision=2)


tex1 = "白　黒　赤"
tex2 = "白　白　黒"
tex3 = "赤　黒"

# 分かち書きされた文書 を格納。
docs = np.array([
    tex1, 
    tex2, 
    tex3
])

In [ ]:
# ベクトル表現化を行う変換器を生成 引数　use_idf=True: tfとidf双方の重みづけ。　 
# token_pattern="(?u)\\b\\w+\\b"⇒特に考えなくて宜しだが　　(?u)は Unicode、\bは空白、\w は文字、+は直前の文字が1文字以上続く意味で、空白で挟まれた文字列を抽出する意味
vectorizer = TfidfVectorizer(use_idf=True, token_pattern="(?u)\\b\\w+\\b")

# 文書をベクトルに変換。 引数には分かち書きされた文章を格納。
vecs = vectorizer.fit_transform(docs)

# 列要素を取得します
print(vectorizer.get_feature_names())

# tf-idf値を格納した行列を取得します
print(vecs.toarray())

['白', '赤', '黒']
[[0.62 0.62 0.48]
 [0.93 0.   0.36]
 [0.   0.79 0.61]]


文字コード順。[こちらの回答](https://aidemy-communication.slack.com/archives/C03ANT80SAE/p1662207583380019?thread_ts=1662203811.211619&cid=C03ANT80SAE)

In [ ]:
# 文字コード順。
print('白：','白'.encode('utf-8'))

# 白： b'\xe7\x99\xbd'
# 赤： b'\xe8\xb5\xa4'
# 黒： b'\xe9\xbb\x92'
# 青： b'\xe9\x9d\x92'

白： b'\xe7\x99\xbd'


その②

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

np.set_printoptions(precision=2) # 桁数を丸める。

tex5 =  "リンゴ リンゴ"
tex6 =  "リンゴ ゴリラ"
tex7 =  "ゴリラ ラッパ"

docs = np.array([tex5,
                 tex6,
                 tex7
                 ])

In [ ]:
# ベクトル表現に変換してください。
vectorizer = TfidfVectorizer(use_idf=True, token_pattern="(?u)\\b\\w+\\b")
vecs = vectorizer.fit_transform(docs)

print(vectorizer.get_feature_names())
print(vecs.toarray())

['ゴリラ', 'ラッパ', 'リンゴ']
[[0.   0.   1.  ]
 [0.71 0.   0.71]
 [0.61 0.8  0.  ]]


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


## ベクトル表現③ COS類似度
 文字を数値化してその文字間の類似度を数値として表現する。つまり、文書のベクトル化⇒文書同士の類似度を解析。

 [Aidemyの教材](https://premium.aidemy.jp/courses/5050/exercises/BJK793LjIlf)

**「1に近い場合は似ていて、0に近いときは似ていない」**⇒これを抑えていこう。

#### 実装

In [ ]:
import numpy as np

def cosine_similarity(v1, v2):
    cos_sim = np.dot(v1, v2) / (np.linalg.norm(v1)*np.linalg.norm(v2))
    return cos_sim

"リンゴ リンゴ",

 "リンゴ ゴリラ",
 
  "ゴリラ ラッパ"
  
  という文の類似度を計算して出力してみましょう。

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

np.set_printoptions(precision=2) # 桁数を丸める。​
tex5 =  "リンゴ リンゴ"
tex6 =  "リンゴ ゴリラ"
tex7 =  "ゴリラ ラッパ"

docs = np.array([tex5, tex6, tex7])

In [ ]:
vectorizer = TfidfVectorizer(use_idf=True, token_pattern="(?u)\\b\\w+\\b")

# 文書をベクトルに変換。 引数には分かち書きされた文章を格納。
vecs = vectorizer.fit_transform(docs)

# tf-idf値を格納した行列を取得します
print(vecs.toarray())

[[0.   0.   1.  ]
 [0.71 0.   0.71]
 [0.61 0.8  0.  ]]


In [ ]:
# cos類似度を求める関数を定義してください。
def cosine_similarity(v1, v2):
    cos_sim = np.dot(v1, v2) / (np.linalg.norm(v1)*np.linalg.norm(v2))
    return cos_sim

In [ ]:
# # 類似度を比較してみましょう。
# print("{:1.3F}".format(cosine_similarity(tex5, tex6)))
# print("{:1.3F}".format(cosine_similarity(tex5, tex7)))
# print("{:1.3F}".format(cosine_similarity(tex6, tex7)))

# 類似度を比較してみましょう。
print("{:1.3F}".format(cosine_similarity(vecs[0], vecs[1])))
print("{:1.3F}".format(cosine_similarity(vecs[0], vecs[2])))
print("{:1.3F}".format(cosine_similarity(vecs[1], vecs[2])))

0.707
0.000
0.428


In [ ]:
# tf-idf値を格納した行列を取得します
vecs = vecs.toarray()
# print(vecs.toarray())​
# cos類似度を求める関数を定義してください。
def cosine_similarity(v1, v2):
    cos_sim = np.dot(v1, v2) / (np.linalg.norm(v1)*np.linalg.norm(v2))
    return cos_sim

# 類似度を比較してみましょう。
print("{:1.3F}".format(cosine_similarity(vecs[0], vecs[1])))
print("{:1.3F}".format(cosine_similarity(vecs[0], vecs[2])))
print("{:1.3F}".format(cosine_similarity(vecs[1], vecs[2])))

0.707
0.000
0.428


# 単語のベクトル表現
文章と何が違うのか分からないけど。

単語をベクトルで表現⇒単語の意味の近さの数値化を行う。同義語の探索等に活用。

これを使用すると…単語と単語の関係性を表現できる。 <font color="red">単語同士の演算</font>も可能。

```
「王様」 - 「男」+ 「女」 = 「女王」
「パリ」 - 「フランス」 + 「日本」 = 「東京」
```

上記をニュアンスやイメージではなく数式として行う事。


## Word2Vec 
単語をベクトル表現するツール。更に次元を圧縮も可能。

例：<font color="red">"男"</font>　という単語と関連性が高い文字列を調べたい。

①ニュース記事をテキストコーパスとして抽出。文章とカテゴリーに分割。
: 「globモジュール」、「with文」、「コーパスの呼び出し」

②取り出した文書を品詞ごとに分割し、リストにする。: 「janome(3)」

③Word2Vecでモデルを生成。: 「Word2Vec（実装）」

④男との関連性が高い語を調べる。: 「Word2Vec（実装）」

## 実装

In [ ]:
# module import
from gensim.models import word2vec

In [ ]:
# モデルを生成：学習に使用するリスト（分かち書きされた文書）を Word2Vec関数 の引数とする
model = word2vec.Word2Vec(リスト, size=a, min_count=b, window=c)
# ただし、a, b, cは数字

・size ：ベクトルの次元数。

・window ：この数の前後の単語を、関連性のある単語と見なして学習を行う。

・min_count ：n回未満登場する単語を破棄。

①「BOW カウント」等で扱ったjanome.tokenizerを使って予め 分かち書き を行います。 

In [ ]:
#実例

import glob
from janome.tokenizer import Tokenizer
from gensim.models import word2vec


def load_livedoor_news_corpus():
    category = {
        "peachy":1,
        "smax":2
    }
    texts  = []
    labels = []
    
    # 以下に説明文のコードの該当箇所を写してください
    #-----------------------------------
    for name, label in category.items():
        files = glob.glob("./5050_nlp_data/{name}/{name}*.txt".format(name=name))
        for file in files:
            with open(file, "r", encoding="utf-8") as f:
                lines = f.read().splitlines() 
                text = "".join(lines[2:])
            texts.append(text)
            labels.append(label)

    return texts, labels
    #-----------------------------------

#texts, labels = load_livedoor_news_corpus()

# 品詞を取り出し「名詞、動詞、形容詞、形容動詞」のリスト作成
def tokenize(part):
    tokens = t.tokenize(",".join(part))
    word = []
    for token in tokens:
        part_of_speech = token.part_of_speech.split(",")[0]
 
        if part_of_speech in ["名詞", "動詞", "形容詞", "形容動詞"]:
            word.append(token.surface)            
    return word

# ラベルと文章に分類
texts, labels = load_livedoor_news_corpus()
t = Tokenizer() # 最初にTokenizerインスタンスを作成する
sentences = tokenize(texts[0:100])  # データ量が多いため制限している
# 以下に回答を作成してください
#word2vec.Word2Vecの引数に関して、size=100, min_count=5, window=15としてください
model = word2vec.Word2Vec([sentences], size=100, min_count=5, window=15)
print(model.most_similar(positive=["男"]))

RuntimeError: ignored

#globモジュール

ファイルやディレクトリを操作するときに便利なモジュール。

名前の確認。

・**ファイル**：文書、写真、音楽など、ユーザーが操作・管理する情報の最小単位。

・**ディレクトリ**：ファイルをまとめる入れ物のこと。

・**パス**：コンピュータ上でファイルやディレクトリの場所のこと。

特殊な文字や文字列を用いて賢くファイルを検索できる点。どういう事？？

特定のファイルを簡単に取得することができる。
https://www.youtube.com/watch?v=86C5NDP1dZs

In [ ]:
import glob

lis = glob.glob("test/sample[0-9].txt")
print(lis)



[]


['test/sample1.txt', 'test/sample2.txt']

例。

①「5050_nlp_dataディレクトリ」の中にある、テキストファイルを出力する方法。

②更にリストのはじめの3つの要素を出力するには…

In [ ]:
import glob

# 5050_nlp_data/sports-watchの中にあるファイルを表示してください
lis = glob.glob("5050_nlp_data/sports-watch/*.txt")#ディレクトリ内のファイルを表示。

print(lis[0:3])

# with分。
テキスト開きでも少しやりました。⇒使いこなせるように。

通常のファイル開き。

**open()**でファイルを開き、　**read()**等でファイルを読み込み、　**close()**でファイルを閉じます。

それよりも楽な with文。

ファイルが自動的にclose()されたり、ファイルを開いている際にエラーが発生しても適切な例外処理が自動的に行われるので、とても便利

In [ ]:
with open("a.text", "r", encoding="utf-8") as f:
  data = f.read()

In [ ]:
# with文を用いて5050_nlp_data/sports-watch上にあるLICENSE.txtを開き、出力してください
with open("5050_nlp_data/sports-watch/LICENSE.txt", "r", encoding="utf-8") as f:
    print(f.read())

# コーパスの取り出し
多分あまり重要じゃない。

**コーパス**： 文書または音声データにある種の情報を与えたデータ の事。

そのようなコーパスを取り出して、分類する事ができます。

In [ ]:
import glob

def load_livedoor_news_corpus():
    category = {
        "dokujo-tsushin": 1,
        "it-life-hack":2,
        "kaden-channel": 3,
        "livedoor-homme": 4,
        "movie-enter": 5,
        "peachy": 6,
        "smax": 7,
        "sports-watch": 8,
        "topic-news":9
    }
    texts  = [] # 全ての記事の文章をここに格納します。
    labels = [] # docsに格納される記事の1〜9のカテゴリーを、ラベルとして扱います。

    # 全てのカテゴリーのディレクトリについて実行します。
    for name, label in category.items():
        # {c_name}にcategory.items()から取得したカテゴリー名c_nameをformatメソッドを用いて埋め込みます。
        files = glob.glob("./5050_nlp_data/{name}/{name}*.txt".format(name=name))
        # 各記事について、本文の情報を以下のように取得します。
        for file in files:
            # with文を用いるため、close()は不要です。
            with open(file, "r", encoding="utf-8") as f:
                # 改行文字で分割
                lines = f.read().splitlines()
                # 分割すると0番目にurl, 1番目に日付、2番目にタイトル、3番目以降に記事本文が記載されています。
                # 記事中の本文を1行にまとめてしまいます。
                text = "".join(lines[2:])

            texts.append(text)
            labels.append(label)

    return texts, labels
    
# 全ての記事の文章データとそのラベル（カテゴリー）を取得します。
texts, labels = load_livedoor_news_corpus()

#  Doc2Vec
Word2Vecを応用した 文章をベクトル化する技術

BOWとの大きな違いは 文の語順 も特徴として考慮に入れることができる⇒BOWとの違い。

実装。

①分かち書き

文章を**janomeのTokenizer**を用い、分かち書きにします。


②TaggedDocument クラスのインスタンスを作成

TaggedDocumentの引数にwords="分かち書きされた各要素", tags=["タグ"]を与える.

⇒TaggedDocument クラスのインスタンスを作成。　
これをTaggedDocumentをリストに格納し、Doc2Vecに渡す。

③Doc2Vec でモデルの生成


In [ ]:
#モデルの学習。
model = Doc2Vec(documents=リスト, min_count=1)

# min_count:最低この回数出現した単語のみを学習に使用

④類似度の出力

In [ ]:
for i in range(4):
  print(model.docvecs.most_similar("d"+str(i)))

実例：

In [ ]:
import glob
from gensim.models.doc2vec import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
from janome.tokenizer import Tokenizer

def load_livedoor_news_corpus():
    category = {
        "dokujo-tsushin": 1,
        "sports-watch": 2,
    }
    texts  = []
    labels = []
    
    # 以下に説明文のコードの該当箇所を写してください
    #-----------------------------------
    for name, label in category.items():
        files = glob.glob("./5050_nlp_data/{name}/{name}*.txt".format(name=name))
        for file in files:
            with open(file, "r", encoding="utf-8") as f:
                lines = f.read().splitlines() 
                text = "".join(lines[2:])
            texts.append(text)
            labels.append(label)

    return texts, labels
    #-----------------------------------

texts, labels = load_livedoor_news_corpus()
# Doc2Vecの処理
token = [] # 各docsの分かち書きした結果を格納するリストです
training_docs = [] # TaggedDocumentを格納するリストです
t = Tokenizer() # 最初にTokenizerインスタンスを作成する
for i in range(4):
    
    # docs[i] を分かち書きして、tokenに格納します
    token.append(t.tokenize(texts[i], wakati=True))
    
    # TaggedDocument クラスのインスタンスを作成して、結果をtraining_docsに格納します
    # タグは "d番号"とします
    training_docs.append(TaggedDocument(words=token[i], tags=["d" + str(i)]))

# 以下に回答を作成してください
model = Doc2Vec(documents=training_docs, min_count=1)

for i in range(4):
    print(model.docvecs.most_similar("d"+str(i)))


ModuleNotFoundError: ignored